<a href="https://colab.research.google.com/github/smokingsnakes83/Truth_model/blob/main/Truth_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [54]:
import pandas as pd
import numpy as np
import google.generativeai as genai
import textwrap

# Used to securely store your API key
from google.colab import userdata

from IPython.display import Markdown
from IPython.display import display

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [55]:
# Obtém a chave de API a partir dos dados do usuário
API_KEY=userdata.get('API_KEY')

# Configura a biblioteca genai com a chave de API obtida
genai.configure(api_key=API_KEY) #Substitua pela sua API_KEY

# Carregando o arquivo CSV em um dataframe pandas

In [56]:
# Carrega o arquivo CSV 'informacoes_ia.csv' em um dataframe pandas
df = pd.read_csv('/content/content_data.csv')

In [57]:
df = pd.DataFrame(df)
df.columns = ['Titulo', 'Conteudo']
#df

# **Model**

In [58]:
# Definindo o modelo na variável model
embed_model = 'models/embedding-001'

# **Função que Gera Embeddings para os Documentos**
#**Documentação Detalhada**
###**Função embed_fn:**<br>
A função **embed_fn** recebe o título e o texto de um documento como entrada.
Ela usa a função genai.embed_content (assumindo que genai é uma biblioteca ou API de embedding) para gerar um vetor de embedding que representa o documento.

O argumento **model** especifica o modelo de embedding a ser usado.

O argumento **task_type='retrieval_document'** indica que o embedding será usado para tarefas de recuperação de documentos.
A função retorna o vetor de embedding do documento.

###**Aplicação da função ao dataframe:**<br>
A linha **df['Embeddings'] = df.apply(lambda row: embed_fn(row['Titulo'], row['Conteudo']), axis=1)** aplica a função embed_fn a cada linha do dataframe df.<br><br>
**df.apply(..., axis=1)** aplica a função a cada linha (eixo 1).<br><br>
A função **lambda lambda row: embed_fn(row['Titulo'], row['Conteudo'])** extrai o título **(row['Titulo'])** e o conteúdo **(row['Conteudo'])** de cada linha e os passa para a função **embed_fn**.<br><br>
O resultado da função **embed_fn (o vetor de embedding)** é armazenado na nova coluna **'Embeddings'** do dataframe.
###**Finalidade:**
Este código gera embeddings para cada documento (representado por título e conteúdo) no dataframe, armazenando os embeddings na coluna 'Embeddings'. Isso permite que os documentos sejam comparados semanticamente usando seus embeddings, por exemplo, para realizar buscas por similaridade ou agrupamento.

In [59]:
def embed_fn(title, text):

  """
  Gera embeddings para documentos usando o modelo especificado.

  Args:
      title: O título do documento.
      text: O corpo do texto do documento.

  Returns:
      Um vetor de embedding representando o documento.
  """

  return genai.embed_content(model=embed_model,
                             content=text,
                             task_type='retrieval_document',
                             title=title)['embedding']

# Aplica a função embed_fn a cada linha do dataframe
df['Embeddings'] = df.apply(lambda row: embed_fn(row['Titulo'], row['Conteudo']), axis=1)

# Exibe o dataframe com a nova coluna 'Embeddings'
#df

#**Função que gera embeddings das consultas**
##**Documentação detalhada:**
###**Objetivo:**<br>
A função ***generate_query*** visa recuperar a informação mais relevante de um conjunto de dados estruturado com base na similaridade semântica com uma consulta fornecida pelo usuário.
###**Funcionamento:**<br>
1. **generate_query:** A função gera um embedding da consulta usando a função *genai.embed_content*.<br>
2. **similarity**: Em seguida, a função calcula a similaridade do cosseno entre o embedding da consulta e os embeddings armazenados no dataframe base.
3. **Identificação da informação mais semelhante:** O índice da informação com maior similaridade é identificado.
Verificação da similaridade: A similaridade é comparada com o limite definido.<br>
4. **Retorno da informação:** Se a similaridade for maior ou igual ao limite, a função retorna a informação correspondente do dataframe.
5. **Mensagem de erro:** Se a similaridade for menor que o limite, a função retorna uma mensagem informando que não foi possível encontrar uma resposta adequada.
#**Utilização:**
A função recebe a consulta do usuário (*query*), o dataframe contendo os embeddings e as informações (*base*), o modelo de embedding (*model*), e o limite de similaridade (*limit*).<br>
A função retorna uma tupla contendo a informação correspondente ou a mensagem de erro, juntamente com o valor da similaridade do cosseno.
###**Observações:**
O código assume que a biblioteca genai está disponível e configurada.
O dataframe base precisa conter as colunas 'Embeddings' e 'Conteudo'.
O limite de similaridade (*limit*) pode ser ajustado para controlar a sensibilidade da busca.



In [60]:
def generate_query(query, base, model, limit=0.66):
  """
  Gera uma consulta semântica e retorna a informação correspondente no dataframe.

  Args:
      query: A consulta do usuário.
      base: O dataframe contendo os embeddings e as informações.
      model: O modelo de embedding.
      limite: Limite de similaridade para considerar uma resposta válida.

  Returns:
      A informação correspondente à consulta ou uma mensagem de erro caso a similaridade seja insuficiente.
  """

  query_embed = genai.embed_content(model=model,
                                    content=query,
                                    task_type='retrieval_query') ['embedding']

  dot_products = np.dot(np.stack(df['Embeddings']), query_embed)
  idx = np.argmax(dot_products)

  # Calcula a similaridade do cosseno
  similarity = dot_products[idx] / (np.linalg.norm(query_embed) * np.linalg.norm(base['Embeddings'][idx]))

  #Verificar se a maior similaridade está acima do limite
  if similarity >= limit:
    print('\nSimilaridade:', similarity)
    return df.iloc[idx]['Conteudo']
  else:
    print('\nSimilaridade:', similarity)
    return 'Sinto muito em não poder ajuda-lo, ainda não fui treinado para dar esta resposta.'

# **Configurações do modelo**

In [61]:
gen_config = {
    'candidate_count': 1,
    'temperature': 0.5
}

safety_config = {
    'harassment': 'block_none',
    'hate': 'block_none',
    'sexual': 'block_none',
    'dangerous': 'block_none'
}

#**Consulta**

In [ ]:
print('Para sair digite (q)')
query = input('Faça sua consulta: ')

while query != 'q':
  passage = generate_query(query, df, embed_model)
  #Markdown(f'###{passage}')

  prompt = f'Reescreva este texto, aja como um especialista no assunto, mas sem adicionar informações que não faça parte do texto: {passage}'

  gen_model = genai.GenerativeModel('gemini-1.5-pro-latest',
                                          generation_config=gen_config,
                                          safety_settings=safety_config)

  response = gen_model.generate_content(prompt, stream=True)
  response.resolve()

  print()

  display(to_markdown(response.text))
  %time

  print('\nPara sair digite (q)')
  query = input('Faça sua consulta: ') # substitua pela sua consulta